In [ ]:
import os
print(os.listdir("/kaggle/input/notebookf30568c6d4"))

In [ ]:
import os
print(os.listdir("/kaggle/input/notebook540950b0f0"))

In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

In [ ]:
import pandas as pd
import numpy as np
import itertools
import os
import sys
from prettytable import PrettyTable
import pickle
from random import shuffle
from sklearn.model_selection import train_test_split
import multiprocessing
from multiprocessing.pool import ThreadPool
from tqdm import tqdm_notebook
print(multiprocessing.cpu_count()," CPU cores")

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["axes.grid"] = False

from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

from PIL import Image
import cv2
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers,Model,Sequential
from tensorflow.keras.layers import Input,GlobalAveragePooling2D,Dropout,Dense,Activation,BatchNormalization,GlobalMaxPooling2D,concatenate,Flatten
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,Callback
from tensorflow.keras.initializers import random_normal
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import binary_crossentropy,categorical_crossentropy,mean_squared_error
from tensorflow.keras import backend as K


In [ ]:
file = open('/kaggle/input/notebookf30568c6d4/df_train_train', 'rb')
df_train_train = pickle.load(file)
file.close()

file = open('/kaggle/input/notebookf30568c6d4/df_train_test', 'rb')
df_train_test = pickle.load(file)
file.close()

In [ ]:
FACTOR = 4
BATCH_SIZE = 8 * FACTOR
IMG_SIZE = 512
EPOCHS = 20
WARMUP_EPOCHS = 5
LEARNING_RATE = 1e-4 * FACTOR
WARMUP_LEARNING_RATE = 1e-3 * FACTOR
HEIGHT = 320
WIDTH = 320
CANAL = 3
N_CLASSES = df_train_train['diagnosis'].nunique()
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5
Train_Cohen_Kappa_Score = 0.832
Train_Accuracy_Score = 0.747
Test_Cohen_Kappa_Score = 0.803
Test_Accuracy_Score = 0.775
LR_WARMUP_EPOCHS_1st = 2
LR_WARMUP_EPOCHS_2nd = 5
STEP_SIZE = len(df_train_train) // BATCH_SIZE
TOTAL_STEPS_1st = WARMUP_EPOCHS * STEP_SIZE
TOTAL_STEPS_2nd = EPOCHS * STEP_SIZE
WARMUP_STEPS_1st = LR_WARMUP_EPOCHS_1st * STEP_SIZE
WARMUP_STEPS_2nd = LR_WARMUP_EPOCHS_2nd * STEP_SIZE

In [ ]:
# ref - https://github.com/umbertogriffo/focal-loss-keras/blob/master/losses.py

'''Below Functions create custom loss functions - Categorical Focal Loss, Binary Focal Loss'''

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [ ]:
def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(loss, axis=1)

    return categorical_focal_loss_fixed

In [ ]:
def plt_show1():
    from matplotlib.pyplot import figure
    img = cv2.imread('/kaggle/input/result/result-1.JPG')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    figure(figsize=(18, 16), dpi=90)
    plt.axis('off')
    plt.grid(b=None)
    plt.imshow(gray)
    plt.show()


In [ ]:

complete_datagen = ImageDataGenerator(rescale=1./255, rotation_range=360,brightness_range=[0.5, 1.5],
                                     zoom_range=[1, 1.2],zca_whitening=True,horizontal_flip=True,
                                     vertical_flip=True,fill_mode='constant')
complete_generator = complete_datagen.flow_from_dataframe(dataframe=df_train_train,
                                                          directory = "/kaggle/input/notebookf30568c6d4/train_images_resized_preprocessed/",
                                                          x_col="file_name",
                                                          target_size=(HEIGHT, WIDTH),
                                                          batch_size=1,
                                                          shuffle=False,
                                                          class_mode=None)
STEP_SIZE_COMPLETE = complete_generator.n//complete_generator.batch_size
print(complete_generator.n)

In [ ]:
test_generator = complete_datagen.flow_from_dataframe(dataframe=df_train_test,
                                                          directory = "/kaggle/input/notebookf30568c6d4/test_images_resized_preprocessed/",
                                                          x_col="file_name",
                                                          target_size=(HEIGHT, WIDTH),
                                                          batch_size=1,
                                                          shuffle=False,
                                                          class_mode=None)
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
print(test_generator.n)

In [ ]:
model = load_model("/kaggle/input/notebook540950b0f0/model_main_training.h5",custom_objects={'categorical_focal_loss_fixed':categorical_focal_loss(alpha=.25, gamma=2),
                                                            'binary_focal_loss_fixed' : binary_focal_loss(alpha=.25, gamma=2)})
train_preds = model.predict_generator(complete_generator, steps=STEP_SIZE_COMPLETE,verbose = 1)



In [ ]:
f = open("/kaggle/working/train_preds","wb")
pickle.dump(train_preds,f)
f.close()

In [ ]:
test_preds = model.predict_generator(test_generator, steps=STEP_SIZE_TEST,verbose = 1)



In [ ]:
f = open("/kaggle/working/test_preds","wb")
pickle.dump(test_preds,f)
f.close()

In [ ]:
print(train_preds[0].shape,train_preds[1].shape,train_preds[2].shape)

train_output_regression = np.array(train_preds[0]).reshape(-1,1)
train_output_classification = np.array(np.argmax(train_preds[1],axis = -1)).reshape(-1,1)
train_output_ordinal_regression = np.array(np.sum(train_preds[2],axis = -1)).reshape(-1,1)

print(train_output_regression.shape,train_output_classification.shape,train_output_ordinal_regression.shape)
X_train = np.hstack((train_output_regression,train_output_classification,train_output_ordinal_regression))
print(X_train.shape)

In [ ]:
print(test_preds[0].shape,test_preds[1].shape,test_preds[2].shape)

test_output_regression = np.array(test_preds[0]).reshape(-1,1)
test_output_classification = np.array(np.argmax(test_preds[1],axis = -1)).reshape(-1,1)
test_output_ordinal_regression = np.array(np.sum(test_preds[2],axis = -1)).reshape(-1,1)

print(test_output_regression.shape,test_output_classification.shape,test_output_ordinal_regression.shape)
X_test = np.hstack((test_output_regression,test_output_classification,test_output_ordinal_regression))
print(X_test.shape)

In [ ]:
model_post = Sequential()
model_post.add(Dense(1, activation='linear', input_shape=(3,)))
model_post.compile(optimizer=optimizers.SGD(lr=LEARNING_RATE), loss='mean_squared_error', metrics=['mean_squared_error'])
# model_post.summary()

In [ ]:
X_df_train_test_np_array = np.array(df_train_test.diagnosis.values).astype(str).astype(float)

In [ ]:
X_df_train_train_np_array = np.array(df_train_train.diagnosis.values).astype(str).astype(float)

In [ ]:
# history = model_post.fit(X_train,np.array(df_train_train.diagnosis.values),
#                          batch_size=BATCH_SIZE,
#                          epochs=50,
#                          verbose=1,
#                          validation_data = (X_test,np.array(df_train_test.diagnosis.values).astype(str).astype(int)))

In [ ]:
history = model_post.fit(X_train,X_df_train_train_np_array,
                         batch_size=BATCH_SIZE,
                         epochs=50,
                         verbose=1,
                         validation_data = (X_test,X_df_train_test_np_array))



In [ ]:
model_post.save("/kaggle/working/model_post_training.h5")


In [ ]:
f = open("/kaggle/working/history_post_training","wb")
pickle.dump(history.history,f)
f.close()

In [ ]:
plt.figure(figsize=(25,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Linear Regression Model Loss - Post Training (50 epochs)')
plt.ylabel('Loss (MSE)')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='best')
plt.xticks(range(1,51))
plt.yticks(range(0,4))
plt.gca().ticklabel_format(axis='both', style='plain', useOffset=False)
plt.show()

In [ ]:
def classify(x):
    if x < 0.5:
        return 0
    elif x < 1.5:
        return 1
    elif x < 2.5:
        return 2
    elif x < 3.5:
        return 3
    return 4

In [ ]:
train_labels = model_post.predict(X_train,batch_size=BATCH_SIZE,verbose = 1)
train_labels = np.apply_along_axis(classify, 1, train_labels)

In [ ]:
test_labels = model_post.predict(X_test,batch_size=BATCH_SIZE,verbose = 1)
test_labels = np.apply_along_axis(classify, 1, test_labels)

In [ ]:
def plot_conf_matrix(true,pred,classes):
    cf = confusion_matrix(true, pred)
    
    df_cm = pd.DataFrame(cf, range(len(classes)), range(len(classes)))
    plt.figure(figsize=(8,5.5))
    sns.set(font_scale=1.4)
    sns.heatmap(df_cm, annot=True, annot_kws={"size": 16},xticklabels = classes ,yticklabels = classes,fmt='g')
    #sns.heatmap(df_cm, annot=True, annot_kws={"size": 16})
    #sns.heatmap(df_cm, annot=True, fmt='.2f', cmap="Blues")
    # plt.show()
    plt.show()

In [ ]:
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
plot_conf_matrix(list(df_train_test['diagnosis'].astype(int)),test_labels,labels)

In [ ]:
cnf_matrix = confusion_matrix(df_train_test['diagnosis'].astype('int'), test_labels)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels, columns=labels)
plt.figure(figsize=(16, 7))
plt_show1()

In [ ]:
Train_Cohen_Kappa_score = cohen_kappa_score(train_labels, df_train_train['diagnosis'].astype('int'), weights='quadratic')
Train_Accuracy_score = accuracy_score(df_train_train['diagnosis'].astype('int'),train_labels)

In [ ]:
print("Train Cohen Kappa score: %.3f" % Train_Cohen_Kappa_Score)
print("Train Accuracy score : %.3f" % Train_Accuracy_Score)

In [ ]:
Test_Cohen_Kappa_score = cohen_kappa_score(test_labels, df_train_test['diagnosis'].astype('int'), weights='quadratic')
Test_Accuracy_score = accuracy_score(df_train_test['diagnosis'].astype('int'),test_labels)

In [ ]:
print("Test Cohen Kappa score: %.3f" % Test_Cohen_Kappa_Score)
print("Test Accuracy score : %.3f" % Test_Accuracy_Score)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        "/kaggle/input/testing2/",
        target_size=(320, 320),
        batch_size=1,
        class_mode='binary',
        shuffle=False)

In [ ]:
test_preds1 = model.predict(test_generator)

In [ ]:
print(test_preds1)

In [ ]:
test_output_regression1 = np.array(test_preds1[0]).reshape(-1,1)
test_output_classification1 = np.array(np.argmax(test_preds1[1],axis = -1)).reshape(-1,1)
test_output_ordinal_regression1 = np.array(np.sum(test_preds1[2],axis = -1)).reshape(-1,1)

print(test_output_regression1.shape,test_output_classification1.shape,test_output_ordinal_regression1.shape)
X_test1 = np.hstack((test_output_regression1,test_output_classification1,test_output_ordinal_regression1))
print(X_test1.shape)

In [ ]:
X1 = model_post.predict(X_test1)

In [ ]:
X11 = np.apply_along_axis(classify, 1, X1)

In [ ]:
print(X11)